In [1]:
import librosa
import numpy as np
import os 


train_folder = 'audio/'
train_files = [os.path.join(train_folder, file) for file in os.listdir(train_folder) if file.endswith('.wav')]

labels = []  
mfccs_list = []  

for file in train_files:
    y, sr = librosa.load(file, sr = None)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)


    labels.append(file)
    mfccs_list.append(mfcc)

    print("Label:", file)
    print("Sample rate:", sr)
    print("Audio shape:", y.shape)
    print("MFCC shape:", mfcc.shape)
    print("Chuyển vị", mfcc.T.shape)
    print(" ")


Label: audio/Bye.wav
Sample rate: 44100
Audio shape: (154624,)
MFCC shape: (12, 303)
Chuyển vị (303, 12)
 
Label: audio/Bye2.wav
Sample rate: 44100
Audio shape: (136192,)
MFCC shape: (12, 267)
Chuyển vị (267, 12)
 
Label: audio/Bye3.wav
Sample rate: 44100
Audio shape: (120832,)
MFCC shape: (12, 237)
Chuyển vị (237, 12)
 
Label: audio/Hello.wav
Sample rate: 44100
Audio shape: (104448,)
MFCC shape: (12, 205)
Chuyển vị (205, 12)
 
Label: audio/Hello2.wav
Sample rate: 44100
Audio shape: (112640,)
MFCC shape: (12, 221)
Chuyển vị (221, 12)
 
Label: audio/Hello3.wav
Sample rate: 44100
Audio shape: (158720,)
MFCC shape: (12, 311)
Chuyển vị (311, 12)
 
